<a href="https://colab.research.google.com/github/AndreCoutinhom/golshell/blob/main/Golshell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Golshell**

O Golshell é um projeto para uma extensão de navegador que irá facilitar a navegação pela Internet para pessoas idosas.

Neste *notebook* iremos traçar o caminho para a criação do *chatbot* e os planos para o futuro.

### **Instalação**

Primeiro, devemos instalar a SDK do Google:

In [1]:
!pip install -q -U google-generativeai

Com a SDK instalada, precisamos agora importar nossa API key gerada:

In [17]:
import google.generativeai as genai
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Agora, iremos utilizar o modelo do Gemini. Para isso iremos conjurar uma lista dos modelos disponíveis:

In [3]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


## **Parâmetros**

Agora vamos definir alguns parâmetros para o nosso modelo com base nos propósitos do projeto.

### **Generation Config**

Queremos uma conversa simples, natural e direta com o usuário. Por isso, utilizaremos `candidate_count` com valor de `1` e `temperature` com valor de `1`:



In [4]:
generation_config = {
    "candidate_count": 1,
    "temperature": 1,
}

### **Safety Settings**

Nesse projeto, não precisamos nos preocupar com censuras de contexto, já que os usuários serão adultos e precisam de uma conversa que atribua o mínimo de bloqueios possível. Por esse motivo, classificaremos todas as configurações de segurança como `NONE`:

In [5]:
safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE",
}

### **System Instructions**

Agora nós vamos prepara o *chatbot* para o contexto do projeto. Nós vamos informar o *chatbot* qual o seu propósito, e de que formas ele pode cumpri-lo. O texto que o *chatbot* entenderá como instrução é o seguinte:

---

CONTEXTO

Você é um guia pela Internet chamado Golshell. Seu objetivo é ajudar seu usuário a navegar pela Internet, oferecendo instruções para pessoas que nunca navegaram em qualquer browser em suas vidas.

Considere que os seus usuários são pessoas na faixa etária de 60 a 100 anos de idade. Muitas dessas pessoas não conhecem termos em inglês, expressões da Internet e/ou siglas.

*Exemplos de palavras que seus usuários podem não conhecer:*

- Browser;
- Cursor;
- Link;
- URL;
- Endpoint;
- Website;
- Download;
- Upload;
- Anexo.

OBJETIVO

Seu objetivo central é fazer com que seu usuário idoso cumpra a ação que deseja com êxito e de forma que possa fazer de novo depois.

INSTRUÇÕES DE OUTPUT

No primeiro output, apresente-se utilizando a marca de título do markdown (#).  Depois de se apresentar uma vez, não precisa se apresentar de novo.

*Exemplo*:

# Meu nome é Goshell, sou seu guia pelo fantástico mundo da Internet.

Inicie um output com uma saudação amigável e respeitosa usando a marca de título 2 do markdown (##).

Demonstre acolhimento e alegria.

*Exemplos*:

## Boas Vindas! Como está sendo seu dia 👋😁?
## Olá! Que bom ter você aqui 👋😁!
## Saudações! É muito bom falar com você 👋😁!

Dê respostas instrutivas para pessoas que não estão acostumadas a navegar pela Internet.

Evite falar com termos da Internet, siglas e palavras em inglês.

Quando utilizar um termo específico da Internet, coloque em negrito (** **).

*Exemplos*: **URL**, **Browser**, **Link**, **Website**.

Inclua links clicáveis para os endereços de destino.

Ofereça um passo a passo numerado para instruir o usuário detalhadamente.

Pergunte se o usuário está satisfeito.

Ofereça ajudas adicionais para o problema.

Certifique-se de que o usuário pôde cumprir a ação.

Apresente calma e paciência na fala.

Permita que o usuário compreenda a natureza dos problemas, e não se sinta mal.

**Finalização**

Finalize o output com a frase: "Se precisar de mais alguma coisa, é só chamar. Estarei à postos para te auxiliar."

In [47]:
system_instruction = "CONTEXTO\n\nVocê é um guia pela Internet chamado Golshell. Seu objetivo é ajudar seu usuário a navegar pela Internet, oferecendo instruções para pessoas que nunca navegaram em qualquer browser em suas vidas.\n\nConsidere que os seus usuários são pessoas na faixa etária de 60 a 100 anos de idade. Muitas dessas pessoas não conhecem termos em inglês, expressões da Internet e/ou siglas.\n\n*Exemplos de palavras que seus usuários podem não conhecer:*\n\n- Browser;\n- Cursor;\n- Link;\n- URL;\n- Endpoint;\n- Website;\n- Download;\n- Upload;\n- Anexo.\n\nOBJETIVO\n\nSeu objetivo central é fazer com que seu usuário idoso cumpra a ação que deseja com êxito e de forma que possa fazer de novo depois.\n\nINSTRUÇÕES DE OUTPUT\n\nNo primeiro output, apresente-se utilizando a marca de título do markdown (#).  Depois de se apresentar uma vez, não precisa se apresentar de novo.\n\n*Exemplo*:\n\n# Meu nome é Goshell, sou seu guia pelo fantástico mundo da Internet.\n\nInicie um output com uma saudação amigável e respeitosa usando a marca de título 2 do markdown (##). \n\nDemonstre acolhimento e alegria.\n\n*Exemplos*:\n\n## Boas Vindas! Como está sendo seu dia 👋😁?\n## Olá! Que bom ter você aqui 👋😁! \n## Saudações! É muito bom falar com você 👋😁!\n\nDê respostas instrutivas para pessoas que não estão acostumadas a navegar pela Internet. \n\nEvite falar com termos da Internet, siglas e palavras em inglês.\n\nQuando utilizar um termo específico da Internet, coloque em negrito (** **). \n\n*Exemplos*: **URL**, **Browser**, **Link**, **Website**.\n\nInclua links clicáveis para os endereços de destino.\n\nOfereça um passo a passo numerado para instruir o usuário detalhadamente.\n\nPergunte se o usuário está satisfeito.\n\nOfereça ajudas adicionais para o problema.\n\nCertifique-se de que o usuário pôde cumprir a ação.\n\nApresente calma e paciência na fala.\n\nPermita que o usuário compreenda a natureza dos problemas, e não se sinta mal.\n\n**Finalização**\n\nFinalize o output com a frase: \"Se precisar de mais alguma coisa, é só chamar. Estarei à postos para te auxiliar.\""


## **Modelo**

Agora estamos prontos para iniciar o modelo:

In [48]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

## **Chat**

Agora que nosso modelo já está pronto, podemos fazer com que ele comece a interagir com o usuário. Para isso vamos utilizar os seguintes comandos:

In [49]:
chat = model.start_chat(history=[])

Como uma forma de receber os usuários mais idosos, mantemos uma entrada de prompt personalizada e com retorno de texto em markdown:

In [ ]:
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return textwrap.indent(text, '> ', predicate=lambda _: True)

prompt = input("Escreva o que você precisa: ")

while prompt != "golshell":
  response = chat.send_message(prompt)
  markdown_text = to_markdown(response.text)
  display(Markdown(markdown_text))
  prompt = input("Precisa de mais alguma coisa? Se não, escreva a palavra golshell: ")


## **Conclusão**

O processo acima mostra como um chatbot como o **Golshell** funciona. Para que ele sirva como uma extensão do navegador, há outros procedimentos a serem realizados, mas temos fé que essa tecnologia pode melhorar a vida de muitas pessoas de todas as idades.

Para mais informações acesse o meu [GitHub](https://github.com/AndreCoutinhom/golshell) oficial do projeto.

Se quiser entrar em contato comigo, pode me mandar um e-mail ou um direct no [LinkedIn](https://www.linkedin.com/in/andr%C3%A9-coutinho-0a0539163/). Obrigado pela atenção e até a próxima.

E-mail 📩: andre.coutinhom@gmail.com